In [1]:
from __future__ import absolute_import, division
import numpy as np 
import senteval
import copy
import operator
import torch
import CNN_Model
PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

import embedding

model = None 
count = 0

/home/rick/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
W0615 12:01:55.543479 140050823137024 deprecation_wrapper.py:119] From /home/rick/Xi/WordEmbeddingEval/SentEval-master-new/official/transformer/model/attention_layer.py:24: The name tf.layers.Layer is deprecated. Please use tf.compat.v1.layers.Layer instead.

W0615 12:01:55.562600 140050823137024 deprecation_wrapper.py:119] From /home/rick/Xi/WordEmbeddingEval/SentEval-master-new/CNN_Model.py:73: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0615 12:01:55.597637 140050823137024 deprecation_wrapper.py:119] From /home/rick/anaconda3/lib/python3.6/site-packages/tensorflow_hub/native_module.py:54: The name tf

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# # SentEval prepare and batcher
# def prepare(params, samples):
#     return

# def batcher(params, batch):
#     sentenceList = []
    
#     for i,sent in enumerate(batch):
#         if len(sent) == 0:
#             batch[i] = ['</p>']
    
#     if params['embed'] == 'Glove':
#         sentences = batch
#     else:
#         sentences = [' '.join(s) for s in batch]
  
#     if params['embed'] == 'Glove':
#         output,_ = embedding.getGloveEmb(sentences)
#     if params['embed'] == 'ELMO':
#         output,_ = embedding.getElmoEmb(sentences)
#     if params['embed'] == 'GPT2':
#         output,_ = embedding.getGPT2Emb(sentences)
#         print(output.shape,output[0].shape)
#     if params['embed'] == 'BERT':
#         output,_ = embedding.getBertEmb(sentences)
#     print(output.shape)

#     for embed in output:

#         maxPool = embed.max(axis = 0,keepdims = True)
#         meanPool = embed.mean(axis = 0,keepdims = True)
#         vec = list(maxPool[0]) + list(meanPool[0])
#         sentenceList.append(vec)
      
#     sentenceList = np.array(sentenceList)
#     print(sentenceList.shape)
#     return sentenceList

# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    sentenceList = []
    
    for i,sent in enumerate(batch):
        if len(sent) == 0:
            batch[i] = ['</p>']
    
    if params['embed'] == 'Glove':
        sentences = batch
    else:
        sentences = [' '.join(s) for s in batch]

    if params['embed'] == 'Glove':
        output,length = embedding.getGloveEmb(sentences,True,-1,"crawl-300d-2M.vec")
        output = np.float32(output)
        print(output.shape)
        output = CNN_Model.getTransformer(params,output)
        
        output_temp = []
        for index,sent in enumerate(output):
            output_temp.append(sent[:length[index]])
        output = output_temp
        
    if params['embed'] == 'ELMO':
        output,_ = embedding.getElmoEmb(sentences)
        output = np.float32(output)
        
        output = CNN_Model.getTransformer(params,output)
        
        
    if params['embed'] == 'GPT2':
        output,length = embedding.getGPT2Emb(sentences,True,-1)
        output = np.float32(output)
        output = CNN_Model.getTransformer(params,output)
        
        output_temp = []

        for index,sent in enumerate(output):
            output_temp.append(sent[:length[index]])
        output = output_temp
        
    if params['embed'] == 'BERT':

        output,length = embedding.getBertEmb(sentences,True,-1)
        print(output.shape,len(length))
        output = np.float32(output)
        output = CNN_Model.getTransformer(params,output)
        output_temp = []

        for index,sent in enumerate(output):
            output_temp.append(sent[:length[index]])
        output = output_temp
        
    if params['embed'] == 'Gensen':
        
        output,length = embedding.getGensenEmb(sentences,True,-1)
        output = np.float32(output)
        output = CNN_Model.getTransformer(params,output)
        output_temp = []

        for index,sent in enumerate(output):
            output_temp.append(sent[:length[index]])
        output = output_temp       
       
    for embed in output:
        maxPool = embed.max(axis = 0,keepdims = True)
        meanPool = embed.mean(axis = 0,keepdims = True)
        vec = list(maxPool[0]) + list(meanPool[0])
        sentenceList.append(vec)
      
    sentenceList = np.array(sentenceList)
    print(sentenceList.shape)
    return sentenceList




In [4]:
tasks = [ 'CR', 'MR' , 'MPQA', 'SUBJ',  'MRPC','TREC','SICKEntailment'] # , 'SICKRelatedness','', 'STSBenchmark'
# 'CR', 'MR' , 'MPQA', 'SUBJ',  'MRPC','TREC',
result = []
seed = 10 
for task in tasks:
    print(task)
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 256,
                                     'tenacity': 5, 'epoch_size': 4,}
    params_senteval['embed'] = 'ELMO' # ELMO, GPT2, BERT
#     params_senteval['embedPath'] = ''
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =    [task]  
    results = se.eval(transfer_tasks)
    print(results)
    result.append(results)

CR
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(63, 2048)
{'CR': {'devacc': 85.3, 'acc': 84.21, 'ndev': 3775, 'ntest': 3775}}
MR
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(128, 2048)
(12

In [ ]:
import tensorflow_hub as hub 
elmo = hub.Module("https://tfhub.dev/google/elmo/2")
batch = ['what the hell','I can\'t do it']
embeddings = elmo(batch,as_dict=True)["elmo"]

In [ ]:
embeddings.eval()

In [ ]:
from bert_serving.client import BertClient
batch = ['what are you thinking','how dare you? Mon!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!']
bc = BertClient()
output = bc.encode(batch)

In [ ]:
output.shape


In [ ]:
batch = ['what are you thinking','how!']
x,_ = embedding.getElmoEmb(batch);x.shape

In [ ]:
result

In [ ]:
batch = ['what are get thinking','how!']
x,_ = embedding.getGPT2Emb(batch);x[1].shape